In [1]:
import logging

import pickle

import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from torch.distributions import constraints
from torch import nn
import pyro
import pyro.distributions as dist
import pyro.optim as optim
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.infer import Predictive
import seaborn as sns
from pyro import poutine
from sklearn import metrics

In [2]:
pyro.set_rng_seed(10)

In [3]:

with open('data_all.pickle', 'rb') as handle:
    data = pickle.load(handle)
print(data.shape)

(1127, 5237)


In [4]:
class PMF_Bayesian(nn.Module):
    # by default our latent space is 50-dimensional
    # and we use 400 hidden units
    def __init__(self, train, dim):
        super().__init__()
        """Build the Probabilistic Matrix Factorization model using pymc3.



        """
        self.dim = dim   
        self.data = train.copy()
        self.n, self.m = self.data.shape
        self.map = None
        self.bounds = (0,1)
        self.losses = None
        self.predictions = None


        # Perform mean value imputation
    
        
        # Low precision reflects uncertainty; prevents overfitting.
        # Set to the mean variance across users and items.
        self.mean_u = (np.mean(self.data, axis=1).mean()) 
        self.mean_v = (np.mean(self.data, axis=0).mean())

        self.std_u =  np.std(self.data, axis=1).mean()
        self.std_v =  np.std(self.data, axis=0).mean()
       
        self.bias = self.data.mean()


    def model(self,  train):

        drug_plate = pyro.plate("drug_latents", self.n, dim= -1) #independent users
        sideeffect_plate = pyro.plate("sideeffect_latents", self.m, dim= -1) #independent items
        U_beta_prior = pyro.sample("U_beta_prior", dist.HalfNormal(self.std_u).expand([self.dim]).to_event(1)) 
        V_beta_prior = pyro.sample("V_beta_prior", dist.HalfNormal(self.std_v).expand([self.dim]).to_event(1))
        U_alpha_prior = pyro.sample("U_alpha_prior", dist.HalfNormal(self.mean_u).expand([self.dim]).to_event(1)) 
        V_alpha_prior = pyro.sample("V_alpha_prior", dist.HalfNormal(self.mean_v, ).expand([self.dim]).to_event(1))
        


        with drug_plate:
            UA = pyro.sample("UA2", dist.Gamma(U_alpha_prior, U_beta_prior).to_event(1))
        
            #UA_int = pyro.sample("UAint", dist.Normal(0., 1.))
        
        with sideeffect_plate:
            VA = pyro.sample("VA2", dist.Gamma(V_alpha_prior, V_beta_prior).to_event(1))    
        
        u2_plate = pyro.plate("u2_plate",self.n, dim=-2)

        with sideeffect_plate, u2_plate: 
            Y = pyro.sample("target", dist.Poisson(UA@VA.T), obs= train ) 
            return Y
            
    def guide(self, train=None):

        d_mean_a = pyro.param('d_alpha_mean', 2*torch.ones(self.dim), constraint=constraints.positive)#*self.user_mean)
        d_std_a = pyro.param('d_alpha_std', 5*torch.ones(self.dim), constraint=constraints.positive)
        d_mean_b = pyro.param('d_beta_mean', 1*torch.ones(self.dim), constraint=constraints.positive)#*self.user_mean)
        d_std_b = pyro.param('d_beta_std', 5*torch.ones(self.dim), constraint=constraints.positive)

        s_mean_a = pyro.param('s_alpha_mean', 2*torch.ones(self.dim), constraint=constraints.positive)#*self.user_mean)
        s_std_a = pyro.param('s_alpha_std', 5*torch.ones(self.dim), constraint=constraints.positive)
        s_mean_b = pyro.param('s_beta_mean', 1*torch.ones(self.dim), constraint=constraints.positive)#*self.user_mean)
        s_std_b = pyro.param('s_beta_std', 5*torch.ones(self.dim), constraint=constraints.positive)

        drug_plate = pyro.plate("drug_latents", self.n, dim= -1) #independent users
        sideeffect_plate = pyro.plate("sideeffect_latents", self.m, dim= -1) #independent items
        U_alpha_prior = pyro.sample("U_alpha_prior", dist.HalfNormal(d_std_a).to_event(1))
        U_beta_prior = pyro.sample("U_beta_prior", dist.HalfNormal( d_std_b).to_event(1))
        V_alpha_prior = pyro.sample("V_alpha_prior", dist.HalfNormal( s_std_a).to_event(1))
        V_beta_prior = pyro.sample("V_beta_prior", dist.HalfNormal(s_std_b).to_event(1))

        with drug_plate: 
            UA = pyro.sample("UA2", dist.Gamma(U_alpha_prior, U_beta_prior).to_event(1))
            # UA_int = pyro.sample("UAint", dist.Normal(int_mean, mov_cov).to_event(1))
       
        
        with sideeffect_plate: 
            VA = pyro.sample("VA2", dist.Gamma(V_alpha_prior, V_beta_prior).to_event(1))

    
    def train_SVI(self,train, nsteps=250, lr = 0.01, lrd = 1):
        logging.basicConfig(format='%(message)s', level=logging.INFO)
        svi = SVI(self.model,
        self.guide,
        optim.ClippedAdam({"lr": lr, "lrd": lrd}),
        loss=Trace_ELBO())
        losses = []
        for step in range(nsteps):
            elbo = svi.step(torch.from_numpy(train).float())
            losses.append(elbo)
            if step % 10 == 0:
                print("Elbo loss: {}".format(elbo))
        self.losses = losses
        #constrained_params = list(pyro.get_param_store().values())
        #PARAMS = [p.unconstrained() for p in constrained_params]
        #print(PARAMS)
        return losses
    
    def sample_predict(self, nsamples=500 , verbose=True):
        predictive_svi = Predictive(self.model, guide=self.guide, num_samples=nsamples)( None)
        if (verbose):
            for k, v in predictive_svi.items():
                print(f"{k}: {tuple(v.shape)}")
        table = predictive_svi["target"].numpy()
        mc_table = table.mean(axis = 0)
        mc_table_std = table.std(axis = 0)
        mc_table[mc_table < self.bounds[1]] = self.bounds[0]
        mc_table[mc_table >= self.bounds[1]] = self.bounds[1]
        self.predictions = mc_table
        
    
    def rmse(self,test):
        low, high = self.bounds
        test_data = test.copy()
        test_data[test_data < high] = low
        test_data[test_data >= high] = high
        sqerror = abs(test_data - self.predictions) ** 2  # squared error array
        mse = sqerror.sum()/(test_data.shape[0]*test_data.shape[1])
        print("PMF MAP training RMSE: %.5f" % np.sqrt(mse))
        fpr, tpr, thresholds = metrics.roc_curve(test_data.astype(int).flatten(),  self.predictions.astype(int).flatten(), pos_label=1)
        metrics.auc(fpr, tpr)
        print("AUC: %.5f" % metrics.auc(fpr, tpr))
        return np.sqrt(mse) , metrics.auc(fpr, tpr)

    def get_predictions(self):
        return self.predictions


In [14]:
class PMF_Bayesian(nn.Module):
    # by default our latent space is 50-dimensional
    # and we use 400 hidden units
    def __init__(self, train, dim):
        super().__init__()
        """Build the Probabilistic Matrix Factorization model using pymc3.



        """
        self.dim = dim   
        self.data = train.copy()
        self.n, self.m = self.data.shape
        self.map = None
        self.bounds = (0,1)
        self.losses = None
        self.predictions = None


        # Perform mean value imputation
    
        
        # Low precision reflects uncertainty; prevents overfitting.
        # Set to the mean variance across users and items.
        self.mean_u = (np.mean(self.data, axis=1).mean()) 
        self.mean_v = (np.mean(self.data, axis=0).mean())

        self.std_u =  np.std(self.data, axis=1).mean()
        self.std_v =  np.std(self.data, axis=0).mean()
       
        self.bias = self.data.mean()


    def model(self, train):

        drug_plate = pyro.plate("drug_latents", self.n, dim= -1) #independent users
        sideeffect_plate = pyro.plate("sideeffect_latents", self.m, dim= -1) #independent items
        U_beta_prior = pyro.sample("U_beta_prior", dist.HalfNormal(self.std_u).expand([self.dim]).to_event(1)) 
        V_beta_prior = pyro.sample("V_beta_prior", dist.HalfNormal(self.std_v).expand([self.dim]).to_event(1))
        U_alpha_prior = pyro.sample("U_alpha_prior", dist.HalfNormal(self.mean_u).expand([self.dim]).to_event(1)) 
        V_alpha_prior = pyro.sample("V_alpha_prior", dist.HalfNormal(self.mean_v, ).expand([self.dim]).to_event(1))
        drug_code = np.arange(self.n)
        se_code =np.arange(self.m)


        with drug_plate:
            UA = pyro.sample("UA2", dist.Gamma(U_alpha_prior, U_beta_prior).to_event(1))
        
            #UA_int = pyro.sample("UAint", dist.Normal(0., 1.))
        
        with sideeffect_plate:
            VA = pyro.sample("VA2", dist.Gamma(V_alpha_prior, V_beta_prior).to_event(1))    
        
        u2_plate = pyro.plate("u2_plate",self.n, dim=-2)

        lamda = UA[drug_code]@VA[se_code].T


        with sideeffect_plate, u2_plate: 
            Y = pyro.sample("target", dist.Poisson(lamda), obs= train ) 
            return Y
            
    def guide(self, train=None):

        d_mean_a = pyro.param('d_alpha_mean', 2*torch.ones(self.dim), constraint=constraints.positive)#*self.user_mean)
        d_std_a = pyro.param('d_alpha_std', 2*torch.ones(self.dim), constraint=constraints.positive)
        d_mean_b = pyro.param('d_beta_mean', 1*torch.ones(self.dim), constraint=constraints.positive)#*self.user_mean)
        d_std_b = pyro.param('d_beta_std', 0.5*torch.ones(self.dim), constraint=constraints.positive)

        s_mean_a = pyro.param('s_alpha_mean', 2*torch.ones(self.dim), constraint=constraints.positive)#*self.user_mean)
        s_std_a = pyro.param('s_alpha_std', 2*torch.ones(self.dim), constraint=constraints.positive)
        s_mean_b = pyro.param('s_beta_mean', 1*torch.ones(self.dim), constraint=constraints.positive)#*self.user_mean)
        s_std_b = pyro.param('s_beta_std', 0.5*torch.ones(self.dim), constraint=constraints.positive)

        drug_plate = pyro.plate("drug_latents", self.n, dim= -1) #independent users
        sideeffect_plate = pyro.plate("sideeffect_latents", self.m, dim= -1) #independent items
        U_alpha_prior = pyro.sample("U_alpha_prior", dist.HalfNormal(d_std_a).to_event(1))
        U_beta_prior = pyro.sample("U_beta_prior", dist.HalfNormal( d_std_b).to_event(1))
        V_alpha_prior = pyro.sample("V_alpha_prior", dist.HalfNormal( s_std_a).to_event(1))
        V_beta_prior = pyro.sample("V_beta_prior", dist.HalfNormal(s_std_b).to_event(1))

        with drug_plate: 
            UA = pyro.sample("UA2", dist.Gamma(U_alpha_prior, U_beta_prior).to_event(1))
            # UA_int = pyro.sample("UAint", dist.Normal(int_mean, mov_cov).to_event(1))
       
        
        with sideeffect_plate: 
            VA = pyro.sample("VA2", dist.Gamma(V_alpha_prior, V_beta_prior).to_event(1))

    
    def train_SVI(self,train, nsteps=250, lr = 0.01, lrd = 1):
        logging.basicConfig(format='%(message)s', level=logging.INFO)
        svi = SVI(self.model,
        self.guide,
        optim.ClippedAdam({"lr": lr, "lrd": lrd}),
        loss=Trace_ELBO())
        losses = []
        for step in range(nsteps):
            elbo = svi.step(torch.from_numpy(train).float())
            losses.append(elbo)
            if step % 10 == 0:
                print("Elbo loss: {}".format(elbo))
        self.losses = losses
        #constrained_params = list(pyro.get_param_store().values())
        #PARAMS = [p.unconstrained() for p in constrained_params]
        #print(PARAMS)
        return losses
    
    def sample_predict(self, nsamples=500 , verbose=True):
        predictive_svi = Predictive(self.model, guide=self.guide, num_samples=nsamples)( None)
        if (verbose):
            for k, v in predictive_svi.items():
                print(f"{k}: {tuple(v.shape)}")
        table = predictive_svi["target"].numpy()
        mc_table = table.mean(axis = 0)
        mc_table_std = table.std(axis = 0)
        mc_table[mc_table < self.bounds[1]] = self.bounds[0]
        mc_table[mc_table >= self.bounds[1]] = self.bounds[1]
        self.predictions = mc_table
        
    
    def rmse(self,test):
        low, high = self.bounds
        test_data = test.copy()
        test_data[test_data < high] = low
        test_data[test_data >= high] = high
        sqerror = abs(test_data - self.predictions) ** 2  # squared error array
        mse = sqerror.sum()/(test_data.shape[0]*test_data.shape[1])
        print("PMF MAP training RMSE: %.5f" % np.sqrt(mse))
        fpr, tpr, thresholds = metrics.roc_curve(test_data.astype(int).flatten(),  self.predictions.astype(int).flatten(), pos_label=1)
        metrics.auc(fpr, tpr)
        print("AUC: %.5f" % metrics.auc(fpr, tpr))
        return np.sqrt(mse) , metrics.auc(fpr, tpr)

    def get_predictions(self):
        return self.predictions


In [5]:
test = PMF_Bayesian(train=data, dim=100)

test.train_SVI(data)
test.sample_predict(1000)
test.rmse(data)
print(test.get_predictions())
print(data)


Elbo loss: 4563501842.166306
Elbo loss: 4855848170.839249
Elbo loss: 1417575133.6302032
Elbo loss: 2307694793.7136536
Elbo loss: 903333629.4626312
Elbo loss: 384427968.3080139
Elbo loss: 258815049.8520813
Elbo loss: 280878195.86831665
Elbo loss: 351424583.7680054
Elbo loss: 316570892.90582275
Elbo loss: 167075685.3709488
Elbo loss: 455859626.98532104
Elbo loss: 110690202.84208679
Elbo loss: 111475583.17886353
Elbo loss: 111289786.4271698
Elbo loss: 119213182.00937653
Elbo loss: 113637966.51865387
Elbo loss: 111749696.16845703
Elbo loss: 118441725.15519714
Elbo loss: 117942827.58818054
Elbo loss: 364935352.5230179
Elbo loss: 131249846.04650116
Elbo loss: 118265534.41383362
Elbo loss: 114628893.55084991
Elbo loss: 112554825.21196747
U_beta_prior: (1000, 1, 1, 100)
V_beta_prior: (1000, 1, 1, 100)
U_alpha_prior: (1000, 1, 1, 100)
V_alpha_prior: (1000, 1, 1, 100)
UA2: (1000, 1, 1127, 100)
VA2: (1000, 1, 5237, 100)
target: (1000, 1127, 5237)
PMF MAP training RMSE: 0.89472
AUC: 0.50000
[[1. 1

In [12]:
#Testing!!!

n,m = data.shape
dim=10
mean_u = 4
mean_v=5
std_v=1
std_u=1
drug_code = np.arange(n)
se_code =np.arange(m)
def model0():
        drug_code = np.arange(n)
        se_code =np.arange(m)
        drug_plate = pyro.plate("drug_latents", n, dim= -1) #independent users
        sideeffect_plate = pyro.plate("sideeffect_latents",m, dim= -1) #independent items
        U_beta_prior = pyro.sample("U_beta_prior", dist.HalfNormal(std_u).expand([dim]).to_event(1)) 
        V_beta_prior = pyro.sample("V_beta_prior", dist.HalfNormal(std_v).expand([dim]).to_event(1))
        U_alpha_prior = pyro.sample("U_alpha_prior", dist.HalfNormal(mean_u).expand([dim]).to_event(1)) 
        V_alpha_prior = pyro.sample("V_alpha_prior", dist.HalfNormal(mean_v, ).expand([dim]).to_event(1))
        


        with drug_plate:
            UA = pyro.sample("UA2", dist.Gamma(U_alpha_prior, U_beta_prior).to_event(1))
        
            #UA_int = pyro.sample("UAint", dist.Normal(0., 1.))
        
        with sideeffect_plate:
            VA = pyro.sample("VA2", dist.Gamma(V_alpha_prior, V_beta_prior).to_event(1))    
        
        u2_plate = pyro.plate("u2_plate",n, dim=-2)

        lamda = UA[drug_code]@VA[se_code].T

        with sideeffect_plate, u2_plate: 
            Y = pyro.sample("target", dist.Poisson(lamda) ) 
            return Y
            

        

        with sideeffect_plate, u2_plate: 
            Y = pyro.sample("target", dist.Poisson(UA@VA.T) ) 
     
            return VA

        
        

      

def guide9():

        d_alpha = pyro.param('d_alpha', 5*torch.ones(n,dim), constraint=constraints.positive)#*self.user_mean)
        d_beta = pyro.param('d_beta', 0.05*torch.ones(n,dim), constraint=constraints.positive)
        # int_mean = pyro.param('int_mean', torch.tensor(1.)*self.user_mean)
        # mov_cov = pyro.param('mov_cov', torch.tensor(1.)*0.1,
            #                  constraint=constraints.positive)
        s_alpha = pyro.param('s_alpha', 5*torch.ones(m,dim), constraint=constraints.positive)#*self.item_mean)
        s_beta = pyro.param('s_beta', 0.05*torch.ones(m,dim), constraint=constraints.positive)
        drug_plate = pyro.plate("drug_latents",n, dim= -1) #independent users
        sideeffect_plate = pyro.plate("sideeffect_latents",m, dim= -1) #independent items
        se_t = pyro.param("sef_int", 0.25*torch.ones(m), constraint=constraints.positive)
        drug_t = pyro.param("drug_int_p", 0.25*torch.ones(n), constraint=constraints.positive)
        u2_plate = pyro.plate("u2_plate", n, dim=-2)
        with u2_plate:
            drug_intercept = pyro.sample("drug_int", dist.HalfNormal(drug_t).to_event(1))

        
        with drug_plate: 
            UA = pyro.sample("UA2", dist.Gamma(d_alpha, d_beta).to_event(1))
            # UA_int = pyro.sample("UAint", dist.Normal(int_mean, mov_cov).to_event(1))
     
        
        with sideeffect_plate: 
            VA = pyro.sample("VA2", dist.Gamma(s_alpha, s_beta).to_event(1))
            sideeffect_intercept =  pyro.sample("sf_int", dist.HalfNormal(se_t))


In [13]:
trace=poutine.trace(model0).get_trace()
trace.compute_log_prob()
print(trace.format_shapes())

          Trace Shapes:               
           Param Sites:               
          Sample Sites:               
      drug_latents dist           |   
                  value      1127 |   
               log_prob           |   
sideeffect_latents dist           |   
                  value      5237 |   
               log_prob           |   
      U_beta_prior dist           | 10
                  value           | 10
               log_prob           |   
      V_beta_prior dist           | 10
                  value           | 10
               log_prob           |   
     U_alpha_prior dist           | 10
                  value           | 10
               log_prob           |   
     V_alpha_prior dist           | 10
                  value           | 10
               log_prob           |   
               UA2 dist      1127 | 10
                  value      1127 | 10
               log_prob      1127 |   
               VA2 dist      5237 | 10
                  value  